## Getting Market Peers GeoLocation

#### Importing Dependencies

In [2]:
import tradingeconomics as te
import pandas as pd
import numpy as np
import pycountry
import plotly.graph_objects as go
import plotly.express as px

from urllib.parse import quote
from tradingeconomics import functions as fn
from tradingeconomics import glob as glob

#### API login


In [152]:
te.login('guest:guest') # Signup for a free API key at https://developer.tradingeconomics.com/

'You are logged in as a guest:guest'

### Importing Data

#### Choose an argument to parameter 'symbol'
    examples:
    - 'PETR3:bs'
    - 'MSFT:us'
    - 'wmt:us'
    - 'hsba:ln'
    

In [146]:
symbol = 'hsba:ln'
df = te.getMarketsPeers(symbols=symbol, output_type='df')

df[['Symbol','Name', 'Country']]


,Symbol,Name,Country
0,ABD:LN,Aberdeen New Dawn,United Kingdom
1,ABN:NA,ABN AMRO Group NV,Netherlands
2,ACA:FP,Credit Agricole,France
3,BAC:US,Bank Of America,United States
4,BARC:LN,Barclays,United Kingdom
5,BBVA:SM,Banco Bilbao Vizcaya Argentaria,Spain
6,BNP:FP,BNP Paribas,France
7,C:US,Citigroup,United States
8,CABK:SM,CaixaBank,Spain
9,CSGN:VX,Credit Suisse,Switzerland


#### 'Group By' to get companies number by country

In [147]:
df4 = df.groupby('Country')['Country'].agg(['count'])
df4

,count
Country,
Austria,2
Belgium,1
Denmark,1
France,3
Germany,1
Italy,3
Netherlands,2
Norway,1
Spain,4


#### Creates a dataframe to receive Country and number of companies

In [148]:
df5 = pd.DataFrame()
df5['Country'] =df4['count'].keys()
df5['Companies'] = df4['count'].values
df5

,Country,Companies
0,Austria,2
1,Belgium,1
2,Denmark,1
3,France,3
4,Germany,1
5,Italy,3
6,Netherlands,2
7,Norway,1
8,Spain,4
9,Sweden,3


### Parsing Country Name to Country alpha_iso3 symbol



In [149]:
countriesWithoutISO = []

missing_countries = {
    'Bolivia' : 'Bolivia, Plurinational State of',
'Cape Verde' : 'Cabo Verde',
'Cayman Islands      '  : 'Cayman Islands',   
'Czech Republic' : 'Czechia',
'European Union' : 'European Union',
"Ivory Coast" : "Côte d'Ivoire",
'Laos' : "Lao People's Democratic Republic",
'Macau' : 'Macao',
'Macedonia' : 'North Macedonia',
'Moldova' : 'Moldova, Republic of',
'Norway              '   : 'Norway',            
'Republic of the Congo' : 'Congo, The Democratic Republic of the',
'Russia' : 'Russian Federation',
'Singapore           '  : 'Singapore',          
'South Korea' : 'Korea, Republic of',
'St Vincent and the Grenadines' : 'Saint Vincent and the Grenadines',
'Swaziland' : 'Eswatini',
'Sweden              ' : 'Sweden',              
'Taiwan' : 'Taiwan, Province of China',
'Tanzania' : 'Tanzania, United Republic of',
'Turkey ': 'Turkey', 
'Venezuela' : 'Venezuela, Bolivarian Republic of',
'Vietnam' : 'Viet Nam',
'United States ': 'United States',
'Brunei': 'Brunei Darussalam',
    'East Timor': 'Timor-Leste',
    'Guinea Bissau': 'Guinea-Bissau',
    'Iran': 'Iran, Islamic Republic of',
    'North Korea': "Korea, Democratic People's Republic of",
    'Palestine': "Palestine, State of",
    'St Kitts and Nevis': 'Saint Kitts and Nevis',
    'St Lucia': 'Saint Lucia',
    'St Vincent And The Grenadines': 'Saint Vincent and the Grenadines'
}

def get_iso3(country_full_name):
            """
                returns country symbol iso alpha 3 format 
            """
            symbol = None
            iso = pycountry.countries.get(name=country_full_name)
            if iso != None:
                symbol = iso.alpha_3
            else:
                if country_full_name not in countriesWithoutISO:
                    countriesWithoutISO.append(country_full_name)
                    print('Failed to get ISO for: ' + country_full_name)
            return symbol



### Treating the data

In [150]:
iso3_countries_names = []
corrected_countries_names_list = df5['Country'].replace(missing_countries)   
df_corrected_countries_names = pd.DataFrame(corrected_countries_names_list)
df_corrected_countries_names['Companies'] = df5['Companies']

for item in df_corrected_countries_names['Country']:
    iso3_countries_names.append(get_iso3(item))

df_corrected_countries_names['iso3'] = iso3_countries_names
df_corrected_countries_names

,Country,Companies,iso3
0,Austria,2,AUT
1,Belgium,1,BEL
2,Denmark,1,DNK
3,France,3,FRA
4,Germany,1,DEU
5,Italy,3,ITA
6,Netherlands,2,NLD
7,Norway,1,NOR
8,Spain,4,ESP
9,Sweden,3,SWE


### Plotting

In [151]:

fig = px.choropleth(df_corrected_countries_names, locations="iso3",title= f"Market Peer for {symbol}" ,
                    color="Companies", 
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()